In [1]:
from utils import get_path, group_cols
from datatable import dt, f, by, as_type

# Descrição

Neste notebook iremos formar a tabela procedimentos.

# Tabelas base

## Localidades

In [2]:
path_locs = get_path('LOCALIDADES', 'localidades.csv.gzip')
df_locs = dt.fread(path_locs)
df_locs

,codigo,cod_municipio,nome_municipio,capital,cod_regiao_saude,nome_regiao_saude,cod_uf,uf,nome_uf,sigla_regiao,nome_regiao,socioecon,latitude,longitude
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CO/GO/52001/520005,520005,Abadia de Goiás,0,52001,Central,52,GO,Goiás,CO,Centro-Oeste,5,−16.7573,−49.4412
1,SD/MG/31052/310010,310010,Abadia dos Dourados,0,31052,Patrocínio Monte Carmelo,31,MG,Minas Gerais,SD,Sudeste,2,−18.4831,−47.3916
2,CO/GO/52011/520010,520010,Abadiânia,0,52011,Pirineus,52,GO,Goiás,CO,Centro-Oeste,3,−16.197,−48.7057
3,SD/MG/31067/310020,310020,Abaeté,0,31067,Sete Lagoas,31,MG,Minas Gerais,SD,Sudeste,3,−19.1551,−45.4444
4,NT/PA/15011/150010,150010,Abaetetuba,0,15011,Tocantins,15,PA,Pará,NT,Norte,1,−1.72183,−48.8788
5,ND/CE/23019/230010,230010,Abaiara,0,23019,19ª Região Brejo Santo,23,CE,Ceará,ND,Nordeste,1,−7.34588,−39.0416
6,ND/BA/29023/290010,290010,Abaíra,0,29023,Seabra,29,BA,Bahia,ND,Nordeste,1,−13.2488,−41.6619
7,ND/BA/29017/290020,290020,Abaré,0,29017,Paulo Afonso,29,BA,Bahia,ND,Nordeste,1,−8.72073,−39.1162
8,SL/PR/41018/410010,410010,Abatiá,0,41018,18ª RS Cornélio Procópio,41,PR,Paraná,SL,Sul,3,−23.3049,−50.3133


## Distâncias

In [3]:
cols_dist = {
  'origem': 'cod_municipio',
  'destino': 'hosp_cod_municipio',
  'distancia (km)': 'distancia (km)',
  'tempo (min)': 'tempo (min)',
  ...: None,
}

In [4]:
path_dist = get_path('DISTANCIAS', 'deslocamentos.csv.gzip')
df_dist = dt.fread(path_dist, columns=cols_dist)
df_dist

,cod_municipio,hosp_cod_municipio,distancia (km),tempo (min)
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,355030,0,0
1,330455,330455,0,0
2,130260,130260,0,0
3,530010,530010,0,0
4,230440,230440,0,0
5,292740,292740,0,0
6,310620,310620,0,0
7,150140,150140,0,0
8,431490,431490,0,0


## SIH

### Colunas da tabela

In [5]:
cols_proc = [
  'ano',
  'parto_normal',
  'cod_municipio',
  'hosp_cod_municipio',
  'cnes',
]

In [6]:
path_sih = get_path('SIH', 'sih.jay')
df_proc = dt.fread(path_sih)
df_proc = df_proc[:, cols_proc]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,310010039,120005,120005,2001020
1,2010,310010039,120040,120040,2000733
2,2010,310010039,120040,120040,2000733
3,2010,310010039,120040,120040,2000733
4,2010,310010039,120040,120040,2000733
5,2010,310010039,120040,120040,2000733
6,2010,310010039,120040,120040,2000733
7,2010,310010039,120005,120005,2001020
8,2010,310010039,120040,120040,2000733


### Tabela agrupada

In [7]:
df_proc = group_cols(df_proc, cols_proc)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,310010039,110001,110001,2679477,141
1,2010,310010039,110001,110002,2494299,1
2,2010,310010039,110001,110013,2808617,2
3,2010,310010039,110001,110020,4001303,2
4,2010,310010039,110001,110028,2495228,9
5,2010,310010039,110002,110002,2494299,507
6,2010,310010039,110002,110011,2808609,5
7,2010,310010039,110002,110012,2495279,1
8,2010,310010039,110002,110020,3970442,1


### Atualizando variável parto normal

In [8]:
# {310010039: 'normal', 411010034: 'cesariano'}
df_proc[:, dt.update(parto_normal = f.parto_normal == 310010039)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,1,110001,110001,2679477,141
1,2010,1,110001,110002,2494299,1
2,2010,1,110001,110013,2808617,2
3,2010,1,110001,110020,4001303,2
4,2010,1,110001,110028,2495228,9
5,2010,1,110002,110002,2494299,507
6,2010,1,110002,110011,2808609,5
7,2010,1,110002,110012,2495279,1
8,2010,1,110002,110020,3970442,1


### Ordenando linhas

In [9]:
df_proc = df_proc.sort(
  f.ano, f.parto_normal, f.cod_municipio, f.hosp_cod_municipio, f.cnes)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49
1,2010,0,110001,110012,2495279,2
2,2010,0,110001,110013,2808617,1
3,2010,0,110001,110018,2496534,1
4,2010,0,110001,110028,2495228,4
5,2010,0,110001,351110,2089327,1
6,2010,0,110002,110002,2494299,375
7,2010,0,110002,110011,2808609,4
8,2010,0,110002,110020,4001303,1


# Gerando tabela

## Colunas origem & destino

In [10]:
def add_info_cols(df, cols, ref):
  df_codes = df[:, cols]
  col_key = cols[0]
  if ref == 'destino':
    col_key = 'hosp_' + col_key
    df_codes.names = {cols[0]: col_key}
  df_codes.names = {c: f'{ref}_{c}' for c in cols[2:]}
  df_codes.names = {cols[1]: ref}
  df_codes.key = col_key
  return df_codes

In [11]:
cols = ['cod_municipio', 'codigo', 'socioecon', 'capital', 'latitude', 'longitude']
df_origem = add_info_cols(df_locs, cols, 'origem')
df_destino = add_info_cols(df_locs, cols, 'destino')
df_proc = df_proc[:, :, dt.join(df_origem)]
df_proc = df_proc[:, :, dt.join(df_destino)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino,destino_socioecon,destino_capital,destino_latitude,destino_longitude
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110001,1,0,−11.9283,−61.9953
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11003/110012,2,0,−10.8777,−61.9322
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11001/110013,1,0,−9.44363,−61.9818
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11002/110018,3,0,−11.672,−61.198
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110028,1,0,−11.7271,−61.7714
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,SD/SP/35151/351110,3,0,−21.1314,−48.977
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11001/110002,1,0,−9.90571,−63.0325
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11003/110011,2,0,−10.4318,−62.4788
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11004/110020,3,1,−8.76077,−63.8999


## CNES concatenado ao destino

In [12]:
df_proc['cnes'] = df_proc[:, as_type(f.cnes, str)]
df_proc[:, dt.update(cnes= ('0000000' + f.cnes)[-7:])]
df_proc[:, dt.update(destino=f.destino + '/' + f.cnes)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino,destino_socioecon,destino_capital,destino_latitude,destino_longitude
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110001/2679477,1,0,−11.9283,−61.9953
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11003/110012/2495279,2,0,−10.8777,−61.9322
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11001/110013/2808617,1,0,−9.44363,−61.9818
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11002/110018/2496534,3,0,−11.672,−61.198
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,NT/RO/11005/110028/2495228,1,0,−11.7271,−61.7714
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,SD/SP/35151/351110/2089327,3,0,−21.1314,−48.977
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11001/110002/2494299,1,0,−9.90571,−63.0325
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11003/110011/2808609,2,0,−10.4318,−62.4788
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,NT/RO/11004/110020/4001303,3,1,−8.76077,−63.8999


## Colunas de distância e tempo

In [13]:
df_dist.key = ['cod_municipio', 'hosp_cod_municipio']
df_proc = df_proc[:, :, dt.join(df_dist)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino_capital,destino_latitude,destino_longitude,distancia (km),tempo (min)
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.9283,−61.9953,0,0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,0,−10.8777,−61.9322,130.77,152.868
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−9.44363,−61.9818,357.511,427.84
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.672,−61.198,114.061,112.283
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,0,−11.7271,−61.7714,48.6099,50.14
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,0,−21.1314,−48.977,2196.35,1852.41
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,−9.90571,−63.0325,0,0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,0,−10.4318,−62.4788,95.4727,140.33
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,1,−8.76077,−63.8999,203.582,188.315


### Distância e tempo ponderado

In [14]:
df_proc[:, dt.update(dist_pond = f['distancia (km)'] * f.count)]
df_proc[:, dt.update(tempo_pond = f['tempo (min)'] * f.count)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,destino_longitude,distancia (km),tempo (min),dist_pond,tempo_pond
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,−61.9953,0,0,0,0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,−61.9322,130.77,152.868,261.541,305.737
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,−61.9818,357.511,427.84,357.511,427.84
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,−61.198,114.061,112.283,114.061,112.283
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,−61.7714,48.6099,50.14,194.44,200.56
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,−48.977,2196.35,1852.41,2196.35,1852.41
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,−63.0325,0,0,0,0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,−62.4788,95.4727,140.33,381.891,561.32
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,−63.8999,203.582,188.315,203.582,188.315


## Coluna socioecon

In [15]:
df_proc['origem_socioecon'] = df_proc[:, as_type(f.origem_socioecon, str)]
df_proc['destino_socioecon'] = df_proc[:, as_type(f.destino_socioecon, str)]
df_proc[:, dt.update(socioecon = f.origem_socioecon + '_' + f.destino_socioecon)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,distancia (km),tempo (min),dist_pond,tempo_pond,socioecon
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,0,0,0,1_1
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,130.77,152.868,261.541,305.737,1_2
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,357.511,427.84,357.511,427.84,1_1
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,114.061,112.283,114.061,112.283,1_3
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,48.6099,50.14,194.44,200.56,1_1
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,2196.35,1852.41,2196.35,1852.41,1_3
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,0,0,0,1_1
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,95.4727,140.33,381.891,561.32,1_2
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,203.582,188.315,203.582,188.315,1_3


## Coluna capitais

In [16]:
df_proc['origem_capital'] = df_proc[:, as_type(f.origem_capital, int)]
df_proc['origem_capital'] = df_proc[:, as_type(f.origem_capital, str)]
df_proc['destino_capital'] = df_proc[:, as_type(f.destino_capital, int)]
df_proc['destino_capital'] = df_proc[:, as_type(f.destino_capital, str)]
df_proc[:, dt.update(capitais = f.origem_capital + '_' + f.destino_capital)]
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,tempo (min),dist_pond,tempo_pond,socioecon,capitais
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,0,0,1_1,0_0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,152.868,261.541,305.737,1_2,0_0
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,427.84,357.511,427.84,1_1,0_0
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,112.283,114.061,112.283,1_3,0_0
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,50.14,194.44,200.56,1_1,0_0
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,1852.41,2196.35,1852.41,1_3,0_0
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,0,0,1_1,0_0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,140.33,381.891,561.32,1_2,0_0
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,188.315,203.582,188.315,1_3,0_1


## Coluna criticidade

In [17]:
crits = list()
origins = df_proc[:, 'origem'].to_list()[0]
destins = df_proc[:, 'destino'].to_list()[0]
for ori, des in zip(origins, destins):
  critic = 0
  trim = ori
  while trim not in des:
    critic += 1
    trim = '/'.join(trim.split('/')[:-1])
  crits.append(critic)
df_proc['criticidade'] = dt.Frame(crits)
df_proc

,ano,parto_normal,cod_municipio,hosp_cod_municipio,cnes,count,origem,origem_socioecon,origem_capital,origem_latitude,…,dist_pond,tempo_pond,socioecon,capitais,criticidade
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,0,110001,110001,2679477,49,NT/RO/11005/110001,1,0,−11.9283,…,0,0,1_1,0_0,0
1,2010,0,110001,110012,2495279,2,NT/RO/11005/110001,1,0,−11.9283,…,261.541,305.737,1_2,0_0,2
2,2010,0,110001,110013,2808617,1,NT/RO/11005/110001,1,0,−11.9283,…,357.511,427.84,1_1,0_0,2
3,2010,0,110001,110018,2496534,1,NT/RO/11005/110001,1,0,−11.9283,…,114.061,112.283,1_3,0_0,2
4,2010,0,110001,110028,2495228,4,NT/RO/11005/110001,1,0,−11.9283,…,194.44,200.56,1_1,0_0,1
5,2010,0,110001,351110,2089327,1,NT/RO/11005/110001,1,0,−11.9283,…,2196.35,1852.41,1_3,0_0,4
6,2010,0,110002,110002,2494299,375,NT/RO/11001/110002,1,0,−9.90571,…,0,0,1_1,0_0,0
7,2010,0,110002,110011,2808609,4,NT/RO/11001/110002,1,0,−9.90571,…,381.891,561.32,1_2,0_0,2
8,2010,0,110002,110020,4001303,1,NT/RO/11001/110002,1,0,−9.90571,…,203.582,188.315,1_3,0_1,2


## Ajustando colunas

In [18]:
cols_order = [
  'ano',
  'parto_normal',
  'origem',
  'destino',
  'distancia (km)',
  'tempo (min)',
  'dist_pond',
  'tempo_pond',
  'criticidade',
  'capitais',
  'socioecon',
  'origem_latitude',
  'origem_longitude',
  'destino_latitude',
  'destino_longitude',
  'count',
]

In [19]:
df_proc = df_proc[:, cols_order]
df_proc

,ano,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0,0,0_0,…,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,261.541,305.737,2,0_0,…,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,357.511,427.84,2,0_0,…,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,114.061,112.283,2,0_0,…,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,194.44,200.56,1,0_0,…,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,2196.35,1852.41,4,0_0,…,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0,0,0_0,…,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,381.891,561.32,2,0_0,…,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,203.582,188.315,2,0_1,…,−9.90571,−63.0325,−8.76077,−63.8999,1


# Biênios (2010/11 & 2018/19)

## Filtrando os anos

In [20]:
df_proc_bienio = df_proc[
  (f.ano == 2010) | (f.ano == 2011) | (f.ano == 2018) |(f.ano == 2019)
, :]
df_proc_bienio

,ano,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0,0,0_0,…,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,261.541,305.737,2,0_0,…,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,357.511,427.84,2,0_0,…,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,114.061,112.283,2,0_0,…,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,194.44,200.56,1,0_0,…,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,2196.35,1852.41,4,0_0,…,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0,0,0_0,…,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,381.891,561.32,2,0_0,…,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,203.582,188.315,2,0_1,…,−9.90571,−63.0325,−8.76077,−63.8999,1


## Atualizando variável ano (biênio)

Valor da variável `bienio`:

- `0 (False) = 2010/11`
- `1 (True) = 2018/19`

In [21]:
df_proc_bienio[:, dt.update(ano = (f.ano == 2018) | (f.ano == 2019))]
df_proc_bienio.names = {'ano': 'bienio'}
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0,0,0_0,…,−11.9283,−61.9953,−11.9283,−61.9953,49
1,0,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,261.541,305.737,2,0_0,…,−11.9283,−61.9953,−10.8777,−61.9322,2
2,0,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,357.511,427.84,2,0_0,…,−11.9283,−61.9953,−9.44363,−61.9818,1
3,0,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,114.061,112.283,2,0_0,…,−11.9283,−61.9953,−11.672,−61.198,1
4,0,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,194.44,200.56,1,0_0,…,−11.9283,−61.9953,−11.7271,−61.7714,4
5,0,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,2196.35,1852.41,4,0_0,…,−11.9283,−61.9953,−21.1314,−48.977,1
6,0,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0,0,0_0,…,−9.90571,−63.0325,−9.90571,−63.0325,375
7,0,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,381.891,561.32,2,0_0,…,−9.90571,−63.0325,−10.4318,−62.4788,4
8,0,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,203.582,188.315,2,0_1,…,−9.90571,−63.0325,−8.76077,−63.8999,1


### Tabela agrupada

In [22]:
df_proc_bienio = df_proc_bienio[:, dt.sum(f.count), by(df_proc_bienio.names[:-1])]
df_proc_bienio

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,964
5,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010510,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,832
6,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010529,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,1831
7,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010537,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3707
8,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010545,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,728


# Exportando tabela

## Ano

In [23]:
output_proc = get_path('GESTANTES', 'procs_ano.csv.gzip')
df_proc.to_csv(output_proc)

In [24]:
dt.fread(output_proc)

,ano,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2010,0,NT/RO/11005/110001,NT/RO/11005/110001/2679477,0,0,0,0,0,0_0,…,−11.9283,−61.9953,−11.9283,−61.9953,49
1,2010,0,NT/RO/11005/110001,NT/RO/11003/110012/2495279,130.77,152.868,261.541,305.737,2,0_0,…,−11.9283,−61.9953,−10.8777,−61.9322,2
2,2010,0,NT/RO/11005/110001,NT/RO/11001/110013/2808617,357.511,427.84,357.511,427.84,2,0_0,…,−11.9283,−61.9953,−9.44363,−61.9818,1
3,2010,0,NT/RO/11005/110001,NT/RO/11002/110018/2496534,114.061,112.283,114.061,112.283,2,0_0,…,−11.9283,−61.9953,−11.672,−61.198,1
4,2010,0,NT/RO/11005/110001,NT/RO/11005/110028/2495228,48.6099,50.14,194.44,200.56,1,0_0,…,−11.9283,−61.9953,−11.7271,−61.7714,4
5,2010,0,NT/RO/11005/110001,SD/SP/35151/351110/2089327,2196.35,1852.41,2196.35,1852.41,4,0_0,…,−11.9283,−61.9953,−21.1314,−48.977,1
6,2010,0,NT/RO/11001/110002,NT/RO/11001/110002/2494299,0,0,0,0,0,0_0,…,−9.90571,−63.0325,−9.90571,−63.0325,375
7,2010,0,NT/RO/11001/110002,NT/RO/11003/110011/2808609,95.4727,140.33,381.891,561.32,2,0_0,…,−9.90571,−63.0325,−10.4318,−62.4788,4
8,2010,0,NT/RO/11001/110002,NT/RO/11004/110020/4001303,203.582,188.315,203.582,188.315,2,0_1,…,−9.90571,−63.0325,−8.76077,−63.8999,1


## Biênio

In [25]:
output_proc_bienio = get_path('GESTANTES', 'procs_bienio.csv.gzip')
df_proc_bienio.to_csv(output_proc_bienio)

In [26]:
dt.fread(output_proc_bienio)

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,964
5,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010510,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,832
6,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010529,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,1831
7,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010537,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3707
8,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010545,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,728
